In [1]:
import xarray as xr
from glob import glob
import numpy as np
from datetime import datetime as dt
from os import listdir, path

from metpy.units import units
from metpy import calc as mpcalc

In [4]:
CMAS_sonde_path = '/Volumes/LtgSSD/CMAS-sondes/'
CMAS_sonde_files = [f for f in sorted(listdir(CMAS_sonde_path)) if f.startswith('GrawSonde') and f.endswith('.nc')]
CMAS_sonde_dts = np.array([dt.strptime('_'.join(f.split('_')[3:5]), '%Y%m%d_%H%M%S.nc') for f in CMAS_sonde_files]).astype('datetime64[s]')
CMAS_sonde_files = np.array([path.join(CMAS_sonde_path, f) for f in CMAS_sonde_files])
# CMAS_day_filter = np.where((CMAS_sonde_dts >= time_start_this_day) & (CMAS_sonde_dts <= time_end_this_day))[0]

In [2]:
# Load the CMAS sondes
CMAS_sonde_path = '/Volumes/LtgSSD/CMAS-sondes/'
CMAS_sonde_files = [f for f in sorted(listdir(CMAS_sonde_path)) if f.startswith('GrawSonde') and f.endswith('.nc')]
print(CMAS_sonde_files[1])

GrawSonde_LidarTruck_RTS_20220529_200522.nc


In [3]:
dt.strptime('_'.join(CMAS_sonde_files[0].split('_')[3:5]), '%Y%m%d_%H%M%S.nc')

datetime.datetime(2022, 5, 29, 17, 46, 35)

In [5]:
cmas_sonde = xr.open_dataset('/Volumes/LtgSSD/CMAS-sondes/GrawSonde_LidarTruck_RTS_20220602_145623.nc')
cmas_sonde 

<xarray.Dataset> Size: 496kB
Dimensions:                    (time: 6205)
Coordinates:
  * time                       (time) float32 25kB 5.378e+04 ... 5.999e+04
Data variables: (12/19)
    time_offset                (time) float32 25kB ...
    pressure                   (time) float32 25kB ...
    temperature                (time) float32 25kB ...
    relative_humidity          (time) float32 25kB ...
    geopotential_height        (time) float32 25kB ...
    dewpoint_temperature       (time) float32 25kB ...
    ...                         ...
    specific_humidity          (time) float32 25kB ...
    wind_speed                 (time) float32 25kB ...
    wind_direction             (time) float32 25kB ...
    longitude                  (time) float32 25kB ...
    latitude                   (time) float32 25kB ...
    geometric_height           (time) float32 25kB ...
Attributes:
    Convention:              GRAWMET Sounding
    DataType:                Real_Time_Sounding, interpolated data
    ReleaseTime:             2022/06/02 14:56:23
    InputFile:               RTS202206021456.txt
    SondeSerialNumber:       20042336.0
    SiteID:                  LidarTruck
    SiteLocation_Latitude:   28.9432 N
    SiteLocation_Longitude:  -95.3009 E
    SiteLocation_Altitude:   3 m
    DataVersion:             1.4, May. 16, 2022
    Created by:              Mariko Oue, mariko.oue@stonybrook.edu

In [6]:
cmas_sonde['pres'] = cmas_sonde.pressure
cmas_sonde['tdry'] = cmas_sonde.temperature
cmas_sonde['dp'] = cmas_sonde.dewpoint_temperature
u, v = mpcalc.wind_components(cmas_sonde.wind_speed * units.meter / units.second, cmas_sonde.wind_direction * units.degree)
cmas_sonde['u_wind'] = u
cmas_sonde['v_wind'] = v
cmas_sonde['alt'] = cmas_sonde.geometric_height

In [16]:
sounding_day = dt.strptime(cmas_sonde.attrs['ReleaseTime'], '%Y/%m/%d %H:%M:%S').replace(hour=0, minute=0, second=0)
sound_day_np = np.datetime64(sounding_day)

In [17]:
times = cmas_sonde.time.data.astype('timedelta64[s]') + sound_day_np